## FermionOperator data structure

In [1]:
import openfermion 
openfermion.__version__

'0.11.1.dev'

In [2]:
from openfermion.ops import FermionOperator

Fermionic ladder operators are similar to qubit ladder operators but distinguished by the canonical fermionic *anticommutation* relations. 

These operators are represented by the `FermionicOperator` data structure, contained in `openfermion.ops`. 

This is implemented as a hash table for fast addition. The keys encode the strings of ladder operators and values store the coefficients. 

The strings are encoded as a tuple of 2-tuples, referred to as "terms tuple." Each ladder operator is represented by a 2-tuple. 

The first element of the 2-tuple is an integer indicated the tensor factor on which the ladder operator acts. The second element is a boolean: 1 means raising and 0 means lowering. $a_8^\dagger$ = (8, 1). 

Note that indices start at 0 and the identity operator is an empty list. 

When initializing a single ladder operator, add the comma after the inner pair because in Python `((a, b)) = (a, b)`.

In [3]:
single_wrong = FermionOperator(((3 ,1)))
print(single_wrong)
single = FermionOperator(((3, 1), ))
print(single)

1.0 [3^]
1.0 [3^]


In [4]:
identity = FermionOperator((),)
print(identity)
zero = FermionOperator()
print(zero)

1.0 []
0


To specify coefficient, provide an optional coefficient argument. DO NOT DO `1.5 * FermionOperator()` because this is slow. 

In [5]:
coeff_initialize = FermionOperator('3^ 1', -1.7)
print(coeff_initialize)

-1.7 [3^ 1]


FermionOperator has only one attribute: `terms`, a dictionary that stores the term tuples. 

In [6]:
my_operator = FermionOperator('4^ 1^ 3 9', 1. + 2.j)
print(my_operator)
print(my_operator.terms)

(1+2j) [4^ 1^ 3 9]
{((4, 1), (1, 1), (3, 0), (9, 0)): (1+2j)}


## Manipulating FermionOperator

Just use normal addition to combine two `FermionOperator` instances. Using `print()` will print each term on a separate line. 

In [7]:
my_op = FermionOperator('4^ 3^ 9 1', 1+2j)
t_2 = FermionOperator('3^ 1', -1.7)
my_op += t_2 # Use += is much more efficient since adds in-place
print(my_op)

-1.7 [3^ 1] +
(1+2j) [4^ 3^ 9 1]


There are many methods that act on the `FermionOperator` data structure. 

In [8]:
from openfermion.utils import commutator, count_qubits, \
    hermitian_conjugated, normal_ordered

print('Get Hermitian conjugate of a FermionOperator, count its qubit,\
        check if normal-ordered') 
term_1 = FermionOperator('4^ 3 3^', 1.+2.j)
print(term_1)
print(hermitian_conjugated(term_1))
print(term_1.is_normal_ordered())
print(count_qubits(term_1))

print('')
print('Normal order the term')
term_2 = normal_ordered(term_1)
print(term_2)
print(term_2.is_normal_ordered())

print('')
print('Compute a commutator of the terms')
print(commutator(term_1, term_2))

Get Hermitian conjugate of a FermionOperator, count its qubit,        check if normal-ordered
(1+2j) [4^ 3 3^]
(1-2j) [3 3^ 4]
False
5

Normal order the term
(1+2j) [4^] +
(-1-2j) [4^ 3^ 3]
True

Compute a commutator of the terms
(-3+4j) [4^ 3 3^ 4^] +
(3-4j) [4^ 3 3^ 4^ 3^ 3] +
(-3+4j) [4^ 3^ 3 4^ 3 3^] +
(3-4j) [4^ 4^ 3 3^]


## QubitOperator

The `QubitOperator` class has almost the same structure as `FermionOperator`, but it stores qubit operators like $X_0 Z_3 Y_4$. The internal representation of this example would be `((0, 'X'), (3, 'Z'), (4, 'Y'))`. 

An important difference between `FermionOperator` and `QubitOperator` is that for the latter, we must **always** sort in order of tensor factor. 

In [9]:
from openfermion.ops import QubitOperator

qubit_op = QubitOperator('X1 Y3 Z3')
print(qubit_op)
print(qubit_op.terms)

op_2 = QubitOperator(((3, 'X'), (4, 'Z')), 3.17)
op_2 -= .77 * qubit_op
print('')
print(op_2)

1j [X1 X3]
{((1, 'X'), (3, 'X')): 1j}

-0.77j [X1 X3] +
3.17 [X3 Z4]


## Jordan-Wigner and Bravyi-Kitaev

In [10]:
from openfermion.transforms import jordan_wigner, bravyi_kitaev
from openfermion.utils import eigenspectrum

# Initialize operator 
ferm_op = FermionOperator('2^ 0', 3.17)
ferm_op += hermitian_conjugated(ferm_op)
print(ferm_op)

# Transform to qubits with JW and print its spectrum 
print('')
jw_op = jordan_wigner(ferm_op)
print(jw_op)
jw_spectrum = eigenspectrum(jw_op)
print(jw_spectrum)

# Transform to qubits with BK and print its spectrum 
print('')
bk_op = bravyi_kitaev(ferm_op)
print(bk_op)
bk_spectrum = eigenspectrum(bk_op)
print(bk_spectrum)

3.17 [0^ 2] +
3.17 [2^ 0]

(1.585+0j) [X0 Z1 X2] +
(1.585+0j) [Y0 Z1 Y2]
[-3.17 -3.17  0.    0.    0.    0.    3.17  3.17]

(1.585+0j) [X0 Y1 Y2] +
(-1.585+0j) [Y0 Y1 X2]
[-3.17 -3.17  0.    0.    0.    0.    3.17  3.17]


Notice that the spectrum is the same for different representations. We can also reverse the JW transform to get a `FermionOperator`. 

In [11]:
from openfermion.transforms import reverse_jordan_wigner

# Map QubitOperator to FermionOperator
mapped_op = normal_ordered(reverse_jordan_wigner(jw_op))

print('Notice this is the same as what we started with:')
print(mapped_op)

Notice this is the same as what we started with:
(3.17+0j) [0^ 2] +
(3.17+0j) [2^ 0]


## Sparse matrices and the Hubbard model

Often, it's useful to obtain a sparse matrix representation of an operator to analyze numerically. `get_sparse_operator()` converts either a `FermionOperator`, `QubitOperator`, or other advanced classes like `InteractionOperator` to a `scipy.sparse.csc` matrix. 

Once we have this representation, we can use many functions from `openfermion.utils` like `get_gap()`, `get_hartree_fock_state()`, and `get_ground_state()`. 

Here we'll use some of these to compute the ground state energy of the Hubbard model. 

In [13]:
from openfermion.hamiltonians import fermi_hubbard 
from openfermion.transforms import get_sparse_operator 
from openfermion.utils import get_ground_state

# Set model 
x_n = 2
y_n = 2 
tunneling = 2. 
coulomb = 1. 
mag_field = 0.5 
chem_potential = 0.25 
periodic = 1
spinless = 1 

# Get fermionic operator from Hubbard model 
hubbard = fermi_hubbard(
    x_n, y_n, tunneling, coulomb, chem_potential, mag_field, 
    periodic, spinless    
)
print(hubbard)

# Get qubit operator from JW 
jw_ham = jordan_wigner(hubbard)
jw_ham.compress()
print('')
print(jw_ham)

# Get scipy.sparse.csc representation 
sparse_op = get_sparse_operator(hubbard)
print('')
print(sparse_op)
print('')
print('My kernel keeps dying when calculating ground state, but I \
        can do up to 10x2 Hubbard on Google Compute!')
# print('Energy of the model is {} in units of T and J.'.format(
#     get_ground_state(sparse_op)[0]
# ))

-0.25 [0^ 0] +
1.0 [0^ 0 1^ 1] +
1.0 [0^ 0 2^ 2] +
-2.0 [0^ 1] +
-2.0 [0^ 2] +
-2.0 [1^ 0] +
-0.25 [1^ 1] +
1.0 [1^ 1 3^ 3] +
-2.0 [1^ 3] +
-2.0 [2^ 0] +
-0.25 [2^ 2] +
1.0 [2^ 2 3^ 3] +
-2.0 [2^ 3] +
-2.0 [3^ 1] +
-2.0 [3^ 2] +
-0.25 [3^ 3]

0.5 [] +
-1.0 [X0 X1] +
-1.0 [X0 Z1 X2] +
-1.0 [Y0 Y1] +
-1.0 [Y0 Z1 Y2] +
-0.375 [Z0] +
0.25 [Z0 Z1] +
0.25 [Z0 Z2] +
-1.0 [X1 Z2 X3] +
-1.0 [Y1 Z2 Y3] +
-0.375 [Z1] +
0.25 [Z1 Z3] +
-1.0 [X2 X3] +
-1.0 [Y2 Y3] +
-0.375 [Z2] +
0.25 [Z2 Z3] +
-0.375 [Z3]

  (1, 1)	(-0.25+0j)
  (2, 1)	(-2+0j)
  (4, 1)	(-2+0j)
  (1, 2)	(-2+0j)
  (2, 2)	(-0.25+0j)
  (8, 2)	(-2+0j)
  (3, 3)	(0.5+0j)
  (6, 3)	(2+0j)
  (9, 3)	(-2+0j)
  (1, 4)	(-2+0j)
  (4, 4)	(-0.25+0j)
  (8, 4)	(-2+0j)
  (5, 5)	(0.5+0j)
  (6, 5)	(-2+0j)
  (9, 5)	(-2+0j)
  (3, 6)	(2+0j)
  (5, 6)	(-2+0j)
  (6, 6)	(-0.5+0j)
  (10, 6)	(-2+0j)
  (12, 6)	(2+0j)
  (7, 7)	(1.25+0j)
  (11, 7)	(-2+0j)
  (13, 7)	(2+0j)
  (2, 8)	(-2+0j)
  (4, 8)	(-2+0j)
  (8, 8)	(-0.25+0j)
  (3, 9)	(-2+0j)
  (5, 9)	(-2+0j)
  (9, 9

## Hamiltonians in the plane wave basis 

